In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import scipy
import scipy.misc as m
import scipy.optimize as o
import os
import io

In [ ]:
config = {
    "countries": ["Italy", "US"],
    "compare-to-us": True,
    "exp-fit-country": True,
    "exp-fit-us": True,
    "label-points": True,
    "stack": False,
    "dataset": "confirmed"
}

In [ ]:
df = pd.read_csv(config["dataset"] + ".csv")

for country in config["countries"]:
    if config['compare-to-us'] == True:
        us_data = df.loc[df["Country/Region"] == "US"].T[4:].T.sum().to_frame()
        us_dates = us_data.T.columns.to_numpy()
        us_data = us_data.T.to_numpy()
        x_us = np.array(list(range(0, len(us_dates))))
        y_us = np.array(list(us_data[0]))
        if config["exp-fit-us"] == True:
            fit_us = o.curve_fit(lambda t,a,b: a*np.exp(b*t),  x_us,  y_us, p0=(4, 0.1), maxfev=1000000000)
            z_us = []
            for xs_us in x_us:
                z_us.append(float(float(fit_us[0][0]) * np.exp(float(fit_us[0][1])*float(xs_us))))
            pred_y_us = np.array(z_us)

    
    italy_data = df.loc[df["Country/Region"] == country].T[4:].T.sum().to_frame()
    if country == "World":
        italy_data = df.loc[df["Lat"] != 0].T[4:].T.sum().to_frame()
    italy_dates = italy_data.T.columns.to_numpy()
    italy_data = italy_data.T.to_numpy()
    x = np.array(list(range(0, len(italy_dates))))
    y = np.array(list(italy_data[0]))
    if config["exp-fit-country"] == True:
        fit = o.curve_fit(lambda t,a,b: a*np.exp(b*t),  x,  y, p0=(4, 0.1), maxfev=1000000000)    
        z = []
        for xs in x:
            z.append(float(float(fit[0][0]) * np.exp(float(fit[0][1])*float(xs))))
        pred_y = np.array(z)
    
#     def f(x3):
#         return float(float(fit[0][0]) * np.exp(float(fit[0][1])*float(x3)))

    ## Let's start plotting the data now that we have all of it.
    plt.title(country + " vs US Coronavirus cases, days since " + str(italy_dates[0]))
    if config["label-points"] == True:    
        for x1,y1 in zip(x,italy_data[0]):
            plt.annotate(y1, # this is the text
                         (x1,y1), # this is the point to label
                         textcoords="offset points", # how to position the text
                         xytext=(0,10), # distance from text to points (x,y)
                         ha='center') # horizontal alignment can be left, right or center
        if config["compare-to-us"] == True:
            for x1_us,y1_us in zip(x_us,us_data[0]):
                plt.annotate(y1_us, # this is the text
                             (x1_us,y1_us), # this is the point to label
                             textcoords="offset points", # how to position the text
                             xytext=(0,10), # distance from text to points (x,y)
                             ha='center') # horizontal alignment can be left, right or center
    plt.scatter(x, italy_data[0])
    plt.plot(x, italy_data[0])
    if config["exp-fit-country"] == True:
        plt.plot(x, pred_y)
    if config["compare-to-us"] == True:
        plt.scatter(x_us, us_data[0])
        plt.plot(x_us, us_data[0])
        if config["exp-fit-us"] == True:
            plt.plot(x_us, pred_y_us)
#     plt.xlabel("Second derivative on latest day: " + str(m.derivative(f, x0=x[len(italy_dates) - 1], dx=1e-6, n=2)))
#     plt.ylabel("US Second derivative on latest day: " + str(m.derivative(f, x0=x_us[len(us_dates) - 1], dx=1e-6, n=2)))
    fig = plt.gcf()
    fig.set_size_inches(20, 12)
    try:
        fig.savefig(config["dataset"] + '_figures/' + country + "/" + italy_dates[len(italy_dates) - 1].replace("/", "-"), dpi=100)
    except FileNotFoundError:
        print("creating")
        os.mkdir(config["dataset"] + "_figures/" + country + "/")
        fig.savefig(config["dataset"] + '_figures/' + country + "/" + italy_dates[len(italy_dates) - 1].replace("/", "-"), dpi=100)
    if config["stack"] == False:
        plt.clf()
        